In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
import argparse
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data.distributed
from torchvision import datasets, transforms
import math
from tqdm import tqdm
import json
import sys
from tinynas.nn.networks import ProxylessNASNets
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import onnx
from pytorch2keras.converter import pytorch_to_keras
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation 
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Convert Pretrained Model from Pytorch to Tensorflow

In [2]:
with open('assets/configs/mcunet-320kb-1mb_imagenet.json') as f:
    config = json.load(f)
model = ProxylessNASNets.build_from_config(config)
model = model.to('cpu')
sd = torch.load('assets/pt_ckpt/mcunet-320kb-1mb_imagenet.pth', map_location='cpu')
model.load_state_dict(sd['state_dict'])
dummy_input = Variable(torch.randn(1,3, 176, 176))
k_model = pytorch_to_keras(model, dummy_input, [(3, 176, 176,)], verbose=True, change_ordering=True)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(3, 176, 176)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight classifier.linear.weight with shape (1000, 160).
DEBUG:onnx2keras:Found weight classifier.linear.bias with shape (1000,).
DEBUG:onnx2keras:Found weight 486 with shape (16, 3, 3, 3).
DEBUG:onnx2keras:Found weight 487 with shape (16,).
DEBUG:onnx2keras:Found weight 489 with shape (16, 1, 3, 3).
DEBUG:onnx2keras:Found weight 490 with shape (16,).
DEBUG:onnx2keras:Found weight 492 with shape (8, 16, 1, 1).
DEBUG:onnx2keras:Found weight 493 with shape (8,).
DEBUG:onnx2keras:Found weight 495 with shape (24

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 485).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 16, 88, 88), dtype=tf.float32, name=None), name='329/Relu6:0', description="created by layer '329'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 488
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 16, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [1, 1], 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 329).
DEBUG:onnx2keras:Check input 1 (name 489).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.


DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 500).
DEBUG:onnx2keras:Check input 1 (name 504).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 505).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 80, 44, 44), dtype=tf.float32, name=None), name='503/BiasAdd:0', description="created by layer '503'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 345
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': True, 'name_policy': None}
DEB

graph(%input_0 : Float(1, 3, 176, 176, strides=[92928, 30976, 176, 1], requires_grad=0, device=cpu),
      %classifier.linear.weight : Float(1000, 160, strides=[160, 1], requires_grad=1, device=cpu),
      %classifier.linear.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %486 : Float(16, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %487 : Float(16, strides=[1], requires_grad=0, device=cpu),
      %489 : Float(16, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %490 : Float(16, strides=[1], requires_grad=0, device=cpu),
      %492 : Float(8, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %493 : Float(8, strides=[1], requires_grad=0, device=cpu),
      %495 : Float(24, 8, 1, 1, strides=[8, 1, 1, 1], requires_grad=0, device=cpu),
      %496 : Float(24, strides=[1], requires_grad=0, device=cpu),
      %498 : Float(24, 1, 7, 7, strides=[49, 49, 7, 1], requires_grad=0, device=cpu),
      %499 : Float(24, str

DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 80, 44, 44), dtype=tf.float32, name=None), name='515/BiasAdd:0', description="created by layer '515'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 357
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 515).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 80, 44, 44), dtype=tf.float32, name=None), name='357/Relu6:0', description="created by layer '357'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 518
DEBUG:onnx2keras:node_pa

...

DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:conv:Number of groups is equal to input channels, use DepthWise convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 200, 11, 11), dtype=tf.float32, name=None), name='587/BiasAdd:0', description="created by layer '587'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 427
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 587).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 200, 11, 11), dtype=tf.float32, name=None), name='427/Relu6:0', description="created by layer '427'")
DEBUG:onnx2keras:

DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 240, 11, 11), dtype=tf.float32, name=None), name='602/BiasAdd:0', description="created by layer '602'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 441
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 602).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 240, 11, 11), dtype=tf.float32, name=None), name='441/Relu6:0', description="created by layer '441'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:on

DEBUG:onnx2keras:node_name: 617
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 453).
DEBUG:onnx2keras:Check input 1 (name 618).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 619).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 96, 6, 6), dtype=tf.float32, name=None), name='617/BiasAdd:0', description="created by layer '617'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DE

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 467).
DEBUG:onnx2keras:Check input 1 (name 633).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 634).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:conv:Number of groups is equal to input channels, use DepthWise convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 384, 6, 6), dtype=tf.float32, name=None), name='632/BiasAdd:0', description="created by layer '632'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2k

DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 644).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 160, 6), dtype=tf.float32, name=None), name='482/Mean:0', description="created by layer '482'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: ReduceMean
DEBUG:onnx2keras:node_name: 483
DEBUG:onnx2keras:node_params: {'axes': [2], 'keepdims': 0, 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 482).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 160), dtype=tf.float32, name=None), name='483/Mean:0', description="created by layer '483'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:o

In [3]:
k_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            [(None, 176, 176, 3) 0                                            
__________________________________________________________________________________________________
485_pad (ZeroPadding2D)         (None, 178, 178, 3)  0           input_0[0][0]                    
__________________________________________________________________________________________________
485 (Conv2D)                    (None, 88, 88, 16)   448         485_pad[0][0]                    
__________________________________________________________________________________________________
329 (ReLU)                      (None, 88, 88, 16)   0           485[0][0]                        
______________________________________________________________________________________________

## Import Dataset

In [4]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 176
batch_size = 50
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

INFO:absl:Load dataset info from /home/nesl/tensorflow_datasets/cifar10/3.0.2
INFO:absl:Reusing dataset cifar10 (/home/nesl/tensorflow_datasets/cifar10/3.0.2)
INFO:absl:Constructing tf.data.Dataset cifar10 for split ['train', 'test'], from /home/nesl/tensorflow_datasets/cifar10/3.0.2


## Transfer Learning

In [5]:
x = k_model.layers[-4].output
x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2, name="top_dropout")(x)
outputs = layers.Dense(10, activation="softmax", name="pred")(x)
model = Model(inputs = k_model.input, outputs = outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            [(None, 176, 176, 3) 0                                            
__________________________________________________________________________________________________
485_pad (ZeroPadding2D)         (None, 178, 178, 3)  0           input_0[0][0]                    
__________________________________________________________________________________________________
485 (Conv2D)                    (None, 88, 88, 16)   448         485_pad[0][0]                    
__________________________________________________________________________________________________
329 (ReLU)                      (None, 88, 88, 16)   0           485[0][0]                        
____________________________________________________________________________________________

In [6]:
model_name = 'mcunet_320_1.h5'
checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(ds_train,
              epochs=100,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/100
1000/1000 [==============================] - 102s 99ms/step - loss: 1.3210 - accuracy: 0.5390 - val_loss: 1.2189 - val_accuracy: 0.5813


DEBUG:h5py._conv:Creating converter from 5 to 3


Epoch 2/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.8419 - accuracy: 0.7067 - val_loss: 0.8696 - val_accuracy: 0.7014
Epoch 3/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.6579 - accuracy: 0.7742 - val_loss: 0.6909 - val_accuracy: 0.7606
Epoch 4/100
1000/1000 [==============================] - 104s 104ms/step - loss: 0.5359 - accuracy: 0.8156 - val_loss: 0.7250 - val_accuracy: 0.7535
Epoch 5/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.4475 - accuracy: 0.8478 - val_loss: 0.5712 - val_accuracy: 0.8049
Epoch 6/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.3757 - accuracy: 0.8706 - val_loss: 0.5538 - val_accuracy: 0.8141
Epoch 7/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.3165 - accuracy: 0.8920 - val_loss: 0.5529 - val_accuracy: 0.8170
Epoch 8/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.2595 - accuracy: 0.9115 - val_loss:

1000/1000 [==============================] - 98s 98ms/step - loss: 0.0342 - accuracy: 0.9882 - val_loss: 0.8453 - val_accuracy: 0.8496
Epoch 58/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.0289 - accuracy: 0.9903 - val_loss: 0.8260 - val_accuracy: 0.8480
Epoch 59/100
1000/1000 [==============================] - 104s 104ms/step - loss: 0.0222 - accuracy: 0.9922 - val_loss: 0.7359 - val_accuracy: 0.8665
Epoch 60/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.0261 - accuracy: 0.9904 - val_loss: 0.9345 - val_accuracy: 0.8382
Epoch 61/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.0273 - accuracy: 0.9906 - val_loss: 0.7409 - val_accuracy: 0.8632
Epoch 62/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.0272 - accuracy: 0.9904 - val_loss: 0.8060 - val_accuracy: 0.8600
Epoch 63/100
1000/1000 [==============================] - 98s 98ms/step - loss: 0.0234 - accuracy: 0.9921 - val_loss: 0.876

In [8]:
new_model = load_model(model_name)
new_model.evaluate(ds_test)

DEBUG:h5py._conv:Creating converter from 3 to 5


200/200 [==============================] - 2s 10ms/step - loss: 0.7037 - accuracy: 0.8765


[0.7037472128868103, 0.8765000104904175]